In [2]:
# ================================
# پیش‌بینی 7 روز آینده با ورودی 1 ماه اخیر (VAR – maxlags کوچک)
# ================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

import os
from datetime import datetime
from statsmodels.tsa.api import VAR

plt.rcParams['figure.figsize'] = (16, 10)
sns.set_style("whitegrid")

# ================================
# مسیرها
# ================================
INPUT_FILE = r"C:\BI\lube_oil_system_data_g11.xlsx"
OUTPUT_EXCEL = r"C:\BI\forecast_main_lube_oil_pump_press_VAR.xlsx"
OUTPUT_PLOTS = r"C:\BI\plots_timeseries_var\\"
LOG_FILE = r"C:\BI\regression_assetid_9286_var_log.txt"
os.makedirs(r"C:\BI", exist_ok=True)
os.makedirs(OUTPUT_PLOTS, exist_ok=True)

def log(msg):
    print(msg)
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {msg}\n")

log("شروع اجرای مدل VAR برای پیش‌بینی 7 روز آینده با ورودی 1 ماه اخیر")

# ================================
# خواندن داده و ساخت datetime
# ================================
df = pd.read_excel(INPUT_FILE)
log(f"داده خوانده شد → {len(df):,} ردیف")

df["datetime"] = pd.to_datetime(df["RecordDate"] + " " + df["RecordTime"], errors="coerce")
df = df.sort_values("datetime").reset_index(drop=True)

# ================================
# نام‌های خوانا
# ================================
friendly_names = {
    "AssetID_8341": "Lube oil filter diffrential pressure",
    "AssetID_8342": "Lube oil tank level",
    "AssetID_8343": "Lube oil tank temperature",
    "AssetID_8344": "Lube oil tank vapor pressure",
    "AssetID_8346": "Over speed pressure",
    "AssetID_9286": "Main lube oil pump press",   # هدف
    "AssetID_9287": "Lube oil line pressure",
}
df.rename(columns=friendly_names, inplace=True)

# ================================
# آماده‌سازی سری‌های زمانی روزانه چندمتغیره
# ================================
target_cols = [
    "Main lube oil pump press",
    "Lube oil line pressure",
    "Lube oil tank temperature",
    "Lube oil tank level"
]

df_multi = df.dropna(subset=target_cols).set_index("datetime")[target_cols]
daily_multi = df_multi.resample("D").mean().interpolate()

if len(daily_multi) < 60:
    raise ValueError("طول سری روزانه کمتر از 60 است؛ برای آموزش پایدار داده کافی نیست.")

log(f"سری چندمتغیره: {len(daily_multi)} روز از {daily_multi.index.min().date()} تا {daily_multi.index.max().date()}")

# ================================
# انتخاب فقط 1 ماه اخیر برای آموزش
# ================================
end_date = daily_multi.index.max()
start_date = end_date - pd.Timedelta(days=30)
train_data = daily_multi.loc[start_date:end_date]

# ================================
# آموزش مدل VAR با maxlags کوچک
# ================================
model = VAR(train_data)
results = model.fit(maxlags=5, ic='aic')   # تغییر مهم: maxlags کوچک‌تر
log("مدل VAR آموزش دید")

# ================================
# پیش‌بینی 7 روز آینده
# ================================
forecast_steps = 7
forecast = results.forecast(train_data.values[-results.k_ar:], steps=forecast_steps)

forecast_df = pd.DataFrame(forecast, columns=train_data.columns,
                           index=pd.date_range(start=end_date + pd.Timedelta(days=1),
                                               periods=forecast_steps, freq="D"))

log("پیش‌بینی 7 روز آینده ساخته شد")

# ================================
# رسم نمودار برای فشار پمپ اصلی
# ================================
plt.figure(figsize=(16, 8))
plt.plot(daily_multi.index, daily_multi["Main lube oil pump press"], label="واقعی (روزانه)", color="steelblue")
plt.plot(forecast_df.index, forecast_df["Main lube oil pump press"], label="پیش‌بینی 7 روز آینده (VAR)", color="green", linewidth=2)
plt.title("Main lube oil pump press - پیش‌بینی 7 روز آینده با ورودی 1 ماه اخیر (VAR – maxlags=5)")
plt.xlabel("تاریخ")
plt.ylabel("فشار")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_PLOTS, "var_7day_forecast.png"), dpi=200, bbox_inches="tight")
plt.close()

# ================================
# ذخیره خروجی در اکسل
# ================================
with pd.ExcelWriter(OUTPUT_EXCEL) as writer:
    daily_multi.to_excel(writer, sheet_name="Daily_History", index=True)
    forecast_df.to_excel(writer, sheet_name="7_Days_Forecast_VAR", index=True)

log("تمام شد! خروجی VAR ذخیره شد")
print("\n" + "="*80)
print("VAR forecast ساخته شد و ذخیره شد")
print(f"فایل اکسل: {OUTPUT_EXCEL}")
print(f"نمودارها: {OUTPUT_PLOTS}")
print("="*80)


شروع اجرای مدل VAR برای پیش‌بینی 7 روز آینده با ورودی 1 ماه اخیر
داده خوانده شد → 10,927 ردیف
سری چندمتغیره: 1690 روز از 2021-04-03 تا 2025-11-17
مدل VAR آموزش دید
پیش‌بینی 7 روز آینده ساخته شد
تمام شد! خروجی VAR ذخیره شد

VAR forecast ساخته شد و ذخیره شد
فایل اکسل: C:\BI\forecast_main_lube_oil_pump_press_VAR.xlsx
نمودارها: C:\BI\plots_timeseries_var\\
